# **Notebook 1 : Prétraitement des Données**

Dans ce notebook, nous préparons les données nécessaires pour l'entraînement des modèles de deep learning. Les principales étapes effectuées sont :

- **Chargement des Données :** Nous importons les images et les légendes associées depuis le jeu de données **Flickr8k**.
- **Sélection d'un Sous-ensemble d'Images :** Nous limitons le jeu de données à **5000 images** pour réduire la complexité computationnelle et faciliter l'entraînement du modèle.
- **Tokenisation du Texte :** Nous convertissons les légendes en séquences numériques en utilisant un tokenizer. Chaque mot est remplacé par son index dans le vocabulaire.
- **Limitation du Vocabulaire :** Nous limitons le vocabulaire aux mots les plus fréquents pour réduire la complexité et améliorer l'efficacité du modèle.
- **Padding des Séquences :** Les séquences sont paddées à une longueur fixe (`MAX_SEQUENCE_LENGTH = 30`) pour assurer une cohérence dans les dimensions des entrées du modèle.
- **Encodage des Labels :** Nous créons des vecteurs de labels pour une classification **multi-label**, où chaque image peut appartenir à plusieurs classes simultanément.
- **Sauvegarde des Données Prétraitées :** Nous sauvegardons les données prétraitées (images, séquences, labels) pour les utiliser dans les notebooks suivants.

**Pourquoi ?**

Un prétraitement adéquat des données est essentiel pour préparer les images et les textes dans un format compatible avec les modèles de deep learning. En limitant le nombre d'images à **5000**, nous réduisons la charge computationnelle et accélérons le temps d'entraînement, tout en conservant une quantité suffisante de données pour que le modèle puisse apprendre efficacement. Cela permet d'assurer que les modèles peuvent apprendre efficacement à partir des données, en réduisant les problèmes liés à la variabilité des entrées.


In [2]:
# Import des bibliothèques nécessaires
import os
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter


In [ ]:
# Chemins des données
data_dir = "../datasets/flickr8k/" 
images_dir = os.path.join(data_dir, "images/") 
captions_file = os.path.join(data_dir, "captions.txt")

# Charger les légendes depuis captions.txt en prenant en compte l'en-tête
captions_df = pd.read_csv(captions_file, sep=",", header=0, names=["image", "caption"]) 

# Afficher les premières lignes
print("Aperçu des données brutes :")
print(captions_df.head())
print(f"Nombre total de légendes : {len(captions_df)}")


Aperçu des données brutes :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  
Nombre total de légendes : 40455


In [4]:
# Fonction pour nettoyer le texte
def clean_caption(caption):
    caption = caption.lower()  # Mettre en minuscule
    caption = re.sub(r"[^a-z0-9 ]", "", caption)  # Retirer les caractères spéciaux
    return caption

# Appliquer le nettoyage sur les légendes
captions_df["cleaned_caption"] = captions_df["caption"].apply(clean_caption)

print("Aperçu des légendes nettoyées :")
print(captions_df[["image", "cleaned_caption"]].head())


Aperçu des légendes nettoyées :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                     cleaned_caption  
0  a child in a pink dress is climbing up a set o...  
1               a girl going into a wooden building   
2    a little girl climbing into a wooden playhouse   
3  a little girl climbing the stairs to her playh...  
4  a little girl in a pink dress going into a woo...  


In [ ]:
# Extraire tous les mots des légendes
all_captions = ' '.join(captions_df["cleaned_caption"]).split()
word_counts = Counter(all_captions)

# Définir le nombre de classes 
NUM_CLASSES = 50 # On prend 50 car cela nous permettra de réduire la taille du vocabulaire et donc accélérer l'entraînement
common_words = [word for word, count in word_counts.most_common(NUM_CLASSES)]

print(f"Top {NUM_CLASSES} mots les plus fréquents comme classes :")
print(common_words)


Top 50 mots les plus fréquents comme classes :
['a', 'in', 'the', 'on', 'is', 'and', 'dog', 'with', 'man', 'of', 'two', 'white', 'black', 'boy', 'are', 'woman', 'girl', 'to', 'wearing', 'at', 'people', 'water', 'red', 'young', 'brown', 'an', 'his', 'blue', 'dogs', 'running', 'through', 'playing', 'while', 'down', 'shirt', 'standing', 'ball', 'little', 'grass', 'child', 'person', 'snow', 'jumping', 'over', 'front', 'three', 'sitting', 'holding', 'field', 'small']


In [6]:
# Fonction pour assigner des classes à une image en fonction de ses légendes
def assign_classes(group, common_words):
    captions = group["cleaned_caption"].tolist()
    classes = set()
    for caption in captions:
        words = caption.split()
        for word in words:
            if word in common_words:
                classes.add(common_words.index(word))  # Utiliser l'index comme label
    return list(classes)

# Limiter les légendes à 5 par image
limited_captions_df = captions_df.groupby("image").head(5).reset_index(drop=True)

print("Nombre d'images uniques après limitation :")
print(len(limited_captions_df["image"].unique()))

# Assignation des classes
class_labels = limited_captions_df.groupby("image").apply(assign_classes, common_words=common_words).reset_index(name='classes')

print("Aperçu des étiquettes de classe assignées :")
print(class_labels.head())


Nombre d'images uniques après limitation :
8091
Aperçu des étiquettes de classe assignées :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1001773457_577c3a7d70.jpg   
2  1002674143_1b742ab4b8.jpg   
3  1003163366_44323f5815.jpg   
4  1007129816_e794419615.jpg   

                                             classes  
0                [0, 1, 2, 4, 37, 39, 9, 16, 17, 25]  
1  [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 14, 19, 2...  
2  [0, 1, 2, 3, 4, 37, 38, 7, 9, 11, 44, 46, 16, ...  
3  [0, 32, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 46, 47...  
4                  [0, 1, 2, 4, 5, 7, 8, 18, 19, 25]  


C:\Users\mathe\AppData\Local\Temp\ipykernel_14320\3283940768.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  class_labels = limited_captions_df.groupby("image").apply(assign_classes, common_words=common_words).reset_index(name='classes')


In [7]:
# Initialiser le tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<unk>")
tokenizer.fit_on_texts(limited_captions_df["cleaned_caption"])

# Convertir les légendes en séquences d'indices
limited_captions_df["caption_sequence"] = tokenizer.texts_to_sequences(
    limited_captions_df["cleaned_caption"]
)

# Padding des séquences
max_caption_length = 30
limited_captions_df["padded_sequence"] = pad_sequences(
    limited_captions_df["caption_sequence"], maxlen=max_caption_length, padding="post"
).tolist()

print("Aperçu des séquences de légendes :")
print(limited_captions_df[["image", "padded_sequence"]].head())


Aperçu des séquences de légendes :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                     padded_sequence  
0  [2, 41, 3, 2, 89, 169, 6, 118, 52, 2, 394, 11,...  
1  [2, 18, 313, 63, 2, 193, 116, 0, 0, 0, 0, 0, 0...  
2  [2, 39, 18, 118, 63, 2, 193, 2430, 0, 0, 0, 0,...  
3  [2, 39, 18, 118, 4, 391, 19, 59, 2430, 0, 0, 0...  
4  [2, 39, 18, 3, 2, 89, 169, 313, 63, 2, 193, 29...  


In [8]:
# Sélectionner 800 images uniques
sampled_images = limited_captions_df["image"].unique()[:5000]

# Filtrer les légendes pour ne garder que celles des 800 images sélectionnées
sampled_captions_df = limited_captions_df[limited_captions_df["image"].isin(sampled_images)]

# Vérifier que chaque image a exactement 5 légendes associées
filtered_grouped_captions = sampled_captions_df.groupby("image").filter(lambda group: len(group) == 5).reset_index(drop=True)

# Obtenir les classes assignées
assigned_classes = class_labels[class_labels['image'].isin(sampled_images)]

# Grouper les données par image
grouped_data = filtered_grouped_captions.groupby("image").apply(
    lambda group: [(os.path.join(images_dir, row["image"]), row["padded_sequence"], assigned_classes[assigned_classes["image"] == row["image"]]["classes"].values[0]) for _, row in group.iterrows()]
).tolist()

# Diviser les images en train/test
train_groups, test_groups = train_test_split(grouped_data, test_size=0.2, random_state=42)

# Reconstituer les données
train_data = [item for group in train_groups for item in group]
test_data = [item for group in test_groups for item in group]

print(f"Nombre d'images uniques dans l'ensemble d'entraînement : {len(train_groups)}")
print(f"Nombre total d'exemples dans l'ensemble d'entraînement : {len(train_data)}")
print(f"Nombre d'images uniques dans l'ensemble de test : {len(test_groups)}")
print(f"Nombre total d'exemples dans l'ensemble de test : {len(test_data)}")


Nombre d'images uniques dans l'ensemble d'entraînement : 4000
Nombre total d'exemples dans l'ensemble d'entraînement : 20000
Nombre d'images uniques dans l'ensemble de test : 1000
Nombre total d'exemples dans l'ensemble de test : 5000


C:\Users\mathe\AppData\Local\Temp\ipykernel_14320\3640787893.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = filtered_grouped_captions.groupby("image").apply(


In [9]:
from tensorflow.keras.utils import to_categorical

# Fonction pour convertir les classes en vecteurs multi-label
def multi_label_encode(classes, num_classes):
    label = np.zeros(num_classes)
    for cls in classes:
        label[cls] = 1
    return label

# Appliquer l'encodage multi-label
train_data_encoded = [(img, seq, multi_label_encode(cls, NUM_CLASSES)) for img, seq, cls in train_data]
test_data_encoded = [(img, seq, multi_label_encode(cls, NUM_CLASSES)) for img, seq, cls in test_data]

# Sauvegarder les données
with open(os.path.join(data_dir, "train_data.pkl"), "wb") as f:
    pickle.dump(train_data_encoded, f)

with open(os.path.join(data_dir, "test_data.pkl"), "wb") as f:
    pickle.dump(test_data_encoded, f)

# Enregistrer le tokenizer et les classes
with open(os.path.join(data_dir, "tokenizer.pkl"), "wb") as f:
    pickle.dump(tokenizer, f)

with open(os.path.join(data_dir, "class_labels.pkl"), "wb") as f:
    pickle.dump(common_words, f)

print("Données prétraitées sauvegardées.")


Données prétraitées sauvegardées.
